In [ ]:
# 코랩기준 우선 설치 후 런타인 다시시작 해야됨
!pip install "scikit_learn==0.22.2.post1"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install catboost
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import random
import os
import numpy as np

In [ ]:
class CFG:
    SEED = 42

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(CFG.SEED) # Seed 고정

## Data Load

In [ ]:
X_train = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/X_train_new.csv')
X_test = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/X_test_new.csv')
y_train = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/y_train_new.csv.csv')
submit = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/sample_submission.csv')

In [ ]:
X_train = train_x.iloc[:,3:]
X_test = test_x.iloc[:,3:]

### 파생변수생성

#### SNP 분할

In [ ]:
np.unique(X_train.iloc[:,1:].values)

array(['A A', 'A G', 'C A', 'C C', 'G A', 'G G'], dtype=object)

In [ ]:
X_list = X_train.iloc[:,1:].columns.to_list()

In [ ]:
for i in X_list:
    X_train[f'{i}_0'] = X_train[f'{i}'].astype('str').apply(lambda x: x.split(' ')).str[0]
for i in X_list:
    X_train[f'{i}_1'] = X_train[f'{i}'].astype('str').apply(lambda x: x.split(' ')).str[1]

In [ ]:
for i in X_list:
    X_test[f'{i}_0'] = X_test[f'{i}'].astype('str').apply(lambda x: x.split(' ')).str[0]
for i in X_list:
    X_test[f'{i}_1'] = X_test[f'{i}'].astype('str').apply(lambda x: x.split(' ')).str[1]

In [ ]:
X_train.iloc[:,16:].head()

,SNP_01_0,SNP_02_0,SNP_03_0,SNP_04_0,SNP_05_0,SNP_06_0,SNP_07_0,SNP_08_0,SNP_09_0,SNP_10_0,...,SNP_06_1,SNP_07_1,SNP_08_1,SNP_09_1,SNP_10_1,SNP_11_1,SNP_12_1,SNP_13_1,SNP_14_1,SNP_15_1
0,G,A,A,G,C,A,A,G,A,G,...,A,A,G,A,G,G,A,A,A,A
1,A,A,C,A,A,A,A,G,A,A,...,G,A,A,A,G,A,A,G,A,A
2,G,G,A,G,C,G,A,G,G,A,...,G,A,A,A,G,A,A,A,A,A
3,A,G,A,G,A,G,G,A,G,A,...,G,G,A,G,G,G,G,G,A,G
4,G,G,C,A,C,A,A,A,A,G,...,A,A,A,A,G,A,A,G,A,A


#### SNP 합

In [ ]:
chrom = pd.DataFrame(snp_info.iloc[:,2])
cm = pd.DataFrame(snp_info.iloc[:,3])
pos = pd.DataFrame(snp_info.iloc[:,4])
chorm_cm = pd.concat([chrom,cm],axis=1)
chorm_pos = pd.concat([chrom,pos],axis=1)
cm_pos = pd.concat([chrom,cm],axis=1)
chrom_cm_pos = pd.concat([chrom,cm,pos],axis=1)
info_list = [chrom,cm,pos,chorm_cm,chorm_pos,cm_pos,chrom_cm_pos]

In [ ]:
predict = pd.DataFrame()
for i in range(len(info_list)):
  model = KMeans(n_clusters=6,algorithm='auto')
  model.fit(info_list[i])
  predict = pd.concat([predict,pd.DataFrame(model.predict(info_list[i]))],axis=1)
predict.columns = ['chrom_clust','cm_clust','pos_clust','chorm_cm_clust','chorm_pos_clust','cm_pos_clust','chrom_cm_pos_clust']
snp_info = pd.concat([snp_info,predict], axis=1)

In [ ]:
chorm_list = []
cm_list = []
pos_list = []
chrom_cm_list = []
chrom_pos_list = []
cm_pos_list = []
chrom_cm_pos_list = []

for i in range(6):
  chorm_list.append(snp_info.query(f'chrom_clust == {i}').SNP_id.to_list())
for i in range(6):
  cm_list.append(snp_info.query(f'cm_clust == {i}').SNP_id.to_list())
for i in range(6):
  pos_list.append(snp_info.query(f'pos_clust == {i}').SNP_id.to_list())
for i in range(6):
  chrom_cm_list.append(snp_info.query(f'chorm_cm_clust == {i}').SNP_id.to_list())
for i in range(6):
  chrom_pos_list.append(snp_info.query(f'chorm_pos_clust == {i}').SNP_id.to_list())
for i in range(6):
  cm_pos_list.append(snp_info.query(f'cm_pos_clust == {i}').SNP_id.to_list())
for i in range(6):
  chrom_cm_pos_list.append(snp_info.query(f'chrom_cm_pos_clust == {i}').SNP_id.to_list())

In [ ]:
all_list = [chorm_list,cm_list,pos_list,chrom_cm_list,chrom_pos_list,cm_pos_list,chrom_cm_pos_list]
all_columns = predict.columns.to_list()

In [ ]:
for j in range(7):
  for i in range(6):
    if len(all_list[j][i]) == 8:
      X_train[f'{all_columns[j]}{i}'] = X_train[all_list[j][i][0]] + ' ' + X_train[all_list[j][i][1]] + ' ' + X_train[all_list[j][i][2]] + ' ' + X_train[all_list[j][i][3]] + ' ' + X_train[all_list[j][i][4]] + ' ' + X_train[all_list[j][i][5]] + ' ' + X_train[all_list[j][i][6]] + ' ' + X_train[all_list[j][i][7]]
    elif len(all_list[j][i]) == 7:
      X_train[f'{all_columns[j]}{i}'] = X_train[all_list[j][i][0]] + ' ' + X_train[all_list[j][i][1]] + ' ' + X_train[all_list[j][i][2]] + ' ' + X_train[all_list[j][i][3]] + ' ' + X_train[all_list[j][i][4]] + ' ' + X_train[all_list[j][i][5]] + ' ' + X_train[all_list[j][i][6]]
    elif len(all_list[j][i]) == 6:
      X_train[f'{all_columns[j]}{i}'] = X_train[all_list[j][i][0]] + ' ' + X_train[all_list[j][i][1]] + ' ' + X_train[all_list[j][i][2]] + ' ' + X_train[all_list[j][i][3]] + ' ' + X_train[all_list[j][i][4]] + ' ' + X_train[all_list[j][i][5]]
    elif len(all_list[j][i]) == 5:
      X_train[f'{all_columns[j]}{i}'] = X_train[all_list[j][i][0]] + ' ' + X_train[all_list[j][i][1]] + ' ' + X_train[all_list[j][i][2]] + ' ' + X_train[all_list[j][i][3]] + ' ' + X_train[all_list[j][i][4]]
    elif len(all_list[j][i]) == 4:
      X_train[f'{all_columns[j]}{i}'] = X_train[all_list[j][i][0]] + ' ' + X_train[all_list[j][i][1]] + ' ' + X_train[all_list[j][i][2]] + ' ' + X_train[all_list[j][i][3]]
    elif len(all_list[j][i]) == 3:
      X_train[f'{all_columns[j]}{i}'] = X_train[all_list[j][i][0]] + ' ' + X_train[all_list[j][i][1]] + ' ' + X_train[all_list[j][i][2]]
    elif len(all_list[j][i]) == 2:
      X_train[f'{all_columns[j]}{i}'] = X_train[all_list[j][i][0]] + ' ' + X_train[all_list[j][i][1]]

In [ ]:
for j in range(7):
  for i in range(6):
    if len(all_list[j][i]) == 8:
      X_test[f'{all_columns[j]}{i}'] = X_test[all_list[j][i][0]] + ' ' + X_test[all_list[j][i][1]] + ' ' + X_test[all_list[j][i][2]] + ' ' + X_test[all_list[j][i][3]] + ' ' + X_test[all_list[j][i][4]] + ' ' + X_test[all_list[j][i][5]] + ' ' + X_test[all_list[j][i][6]] + ' ' + X_test[all_list[j][i][7]]
    elif len(all_list[j][i]) == 7:
      X_test[f'{all_columns[j]}{i}'] = X_test[all_list[j][i][0]] + ' ' + X_test[all_list[j][i][1]] + ' ' + X_test[all_list[j][i][2]] + ' ' + X_test[all_list[j][i][3]] + ' ' + X_test[all_list[j][i][4]] + ' ' + X_test[all_list[j][i][5]] + ' ' + X_test[all_list[j][i][6]]
    elif len(all_list[j][i]) == 6:
      X_test[f'{all_columns[j]}{i}'] = X_test[all_list[j][i][0]] + ' ' + X_test[all_list[j][i][1]] + ' ' + X_test[all_list[j][i][2]] + ' ' + X_test[all_list[j][i][3]] + ' ' + X_test[all_list[j][i][4]] + ' ' + X_test[all_list[j][i][5]]
    elif len(all_list[j][i]) == 5:
      X_test[f'{all_columns[j]}{i}'] = X_test[all_list[j][i][0]] + ' ' + X_test[all_list[j][i][1]] + ' ' + X_test[all_list[j][i][2]] + ' ' + X_test[all_list[j][i][3]] + ' ' + X_test[all_list[j][i][4]]
    elif len(all_list[j][i]) == 4:
      X_test[f'{all_columns[j]}{i}'] = X_test[all_list[j][i][0]] + ' ' + X_test[all_list[j][i][1]] + ' ' + X_test[all_list[j][i][2]] + ' ' + X_test[all_list[j][i][3]]
    elif len(all_list[j][i]) == 3:
      X_test[f'{all_columns[j]}{i}'] = X_test[all_list[j][i][0]] + ' ' + X_test[all_list[j][i][1]] + ' ' + X_test[all_list[j][i][2]]
    elif len(all_list[j][i]) == 2:
      X_test[f'{all_columns[j]}{i}'] = X_test[all_list[j][i][0]] + ' ' + X_test[all_list[j][i][1]]

In [ ]:
X_train.iloc[:,46:].head()

,chrom_clust0,chrom_clust1,cm_clust0,cm_clust2,pos_clust0,pos_clust3,pos_clust4,chorm_cm_clust0,chorm_cm_clust2,chorm_cm_clust5,chorm_pos_clust0,chorm_pos_clust1,chorm_pos_clust4,cm_pos_clust0,cm_pos_clust2,cm_pos_clust5,chrom_cm_pos_clust0,chrom_cm_pos_clust1,chrom_cm_pos_clust3
0,A A A A A A,A G A A G A C A A A A A G G A A,G A C A A A A A G G,G G A A G G A A A A A A,G A C A A A A A A A A A,G G A A G G,G G A A A A,G A C A A A A A G G A A,G G A A A A A A,G G A A,G G A A G G,G A C A A A A A A A A A,G G A A A A,G G A A A A A A,G A C A A A A A G G A A,G G A A,G G A A A A,G G A A G G,G A C A A A A A A A A A
1,G A G G A A,A G C A A A A A A G A A G A A A,A A A A A G A A G A,A G C A A G G A G G A A,A A A A A G A A G A G G,A G C A A G,G A A A A A,A A A A A G A A G A A A,A G G A G G A A,A G C A,A G C A A G,A A A A A G A A G A G G,G A A A A A,A G G A G G A A,A A A A A G A A G A A A,A G C A,G A A A A A,A G C A A G,A A A A A G A A G A G G
2,A A A A A A,G G A A G A C C G G A A G A G A,G A C C G G A A G A,G G A A A G A A A A A A,G A C C G G A A A A A A,G G A A A G,G A G A A A,G A C C G G A A G A G A,A G A A A A A A,G G A A,G G A A A G,G A C C G G A A A A A A,G A G A A A,A G A A A A A A,G A C C G G A A G A G A,G G A A,G A G A A A,G G A A A G,G A C C G G A A A A A A
3,G G G G A A,G G A A G A A A G G G G A A G G,G A A A G G G G A A,A A A A A G G G G G A A,G A A A G G G G G G G G,A A A A A G,A A G G A A,G A A A G G G G A A G G,A G G G G G A A,A A A A,A A A A A G,G A A A G G G G G G G G,A A G G A A,A G G G G G A A,G A A A G G G G A A G G,A A A A,A A G G A A,A A A A A G,G A A A G G G G G G G G
4,A A A G A A,G G C C A A C C A A A A A A A A,A A C C A A A A A A,G G C C G G A A A G A A,A A C C A A A A A A A G,G G C C G G,A A A A A A,A A C C A A A A A A A A,G G A A A G A A,G G C C,G G C C G G,A A C C A A A A A A A G,A A A A A A,G G A A A G A A,A A C C A A A A A A A A,G G C C,A A A A A A,G G C C G G,A A C C A A A A A A A G


#### SNP A,C,G 개수

In [ ]:
from collections import Counter

In [ ]:
X_train['A_count'] = Counter(X_train.iloc[:,16:46].iloc[0,:])['A']
X_train['C_count'] = Counter(X_train.iloc[:,16:46].iloc[0,:])['C']
X_train['G_count'] = Counter(X_train.iloc[:,16:46].iloc[0,:])['G']

In [ ]:
X_test['A_count'] = Counter(X_test.iloc[:,16:46].iloc[0,:])['A']
X_test['C_count'] = Counter(X_test.iloc[:,16:46].iloc[0,:])['C']
X_test['G_count'] = Counter(X_test.iloc[:,16:46].iloc[0,:])['G']

In [ ]:
for i in range(len(X_train.iloc[:,16:].index)):
    X_train['A_count'].iloc[i] = Counter(X_train.iloc[:,16:46].iloc[i,:])['A']
for i in range(len(X_train.iloc[:,16:].index)):
    X_train['C_count'].iloc[i] = Counter(X_train.iloc[:,16:46].iloc[i,:])['C']
for i in range(len(X_train.iloc[:,16:].index)):
    X_train['G_count'].iloc[i] = Counter(X_train.iloc[:,16:46].iloc[i,:])['G']


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
for i in range(len(X_test.iloc[:,16:].index)):
    X_test['A_count'].iloc[i] = Counter(X_test.iloc[:,16:46].iloc[i,:])['A']
for i in range(len(X_test.iloc[:,16:].index)):
    X_test['C_count'].iloc[i] = Counter(X_test.iloc[:,16:46].iloc[i,:])['C']
for i in range(len(X_test.iloc[:,16:].index)):
    X_test['G_count'].iloc[i] = Counter(X_test.iloc[:,16:46].iloc[i,:])['G']

In [ ]:
X_train['A+C_count'] = X_train['A_count']+X_train['C_count']
X_train['A+G_count'] = X_train['A_count']+X_train['G_count']
X_train['C+G_count'] = X_train['C_count']+X_train['G_count']

In [ ]:
X_test['A+C_count'] = X_test['A_count']+X_test['C_count']
X_test['A+G_count'] = X_test['A_count']+X_test['G_count']
X_test['C+G_count'] = X_test['C_count']+X_test['G_count']

In [ ]:
X_train.iloc[:,65:].head()

,A_count,C_count,G_count,A+C_count,A+G_count,C+G_count
0,20,1,9,21,29,10
1,21,1,8,22,29,9
2,18,2,10,20,28,12
3,12,0,18,12,30,18
4,18,4,8,22,26,12


#### trait 문자형 변환

In [ ]:
X_train['trait'] = X_train['trait'].astype(str)
X_test['trait'] = X_test['trait'].astype(str)

In [ ]:
Cat_list = X_train.select_dtypes(include='object').columns.to_list()
Num_list = X_train.select_dtypes(exclude='object').columns.to_list()

In [ ]:
X_train.shape, X_test.shape, y_train.shape

((262, 71), (175, 71), (262,))

### Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
stscaler = StandardScaler()
X_train[Num_list] = stscaler.fit_transform(X_train[Num_list])
X_test[Num_list] = stscaler.transform(X_test[Num_list])

In [ ]:
X_train.head()

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,...,cm_pos_clust5,chrom_cm_pos_clust0,chrom_cm_pos_clust1,chrom_cm_pos_clust3,A_count,C_count,G_count,A+C_count,A+G_count,C+G_count
0,2,G G,A G,A A,G A,C A,A A,A A,G G,A A,...,G G A A,G G A A A A,G G A A G G,G A C A A A A A A A A A,0.986104,-1.002583,-0.663529,0.663529,1.002583,-0.986104
1,2,A G,A G,C A,A A,A A,A G,A A,G A,A A,...,A G C A,G A A A A A,A G C A A G,A A A A A G A A G A G G,1.270327,-1.002583,-0.938165,0.938165,1.002583,-1.270327
2,2,G G,G G,A A,G A,C C,G G,A A,G A,G A,...,G G A A,G A G A A A,G G A A A G,G A C C G G A A A A A A,0.417657,-0.050856,-0.388893,0.388893,0.050856,-0.417657
3,1,A A,G G,A A,G A,A A,G G,G G,A A,G G,...,A A A A,A A G G A A,A A A A A G,G A A A G G G G G G G G,-1.287685,-1.954310,1.808195,-1.808195,1.954310,1.287685
4,2,G G,G G,C C,A A,C C,A A,A A,A A,A A,...,G G C C,A A A A A A,G G C C G G,A A C C A A A A A A A G,0.417657,1.852599,-0.938165,0.938165,-1.852599,-0.417657


### Encoding

In [ ]:
def catboost_encode_multiclass(X,X_t,y):
    y=y.astype(str)
    enc=ce.OneHotEncoder().fit(y)
    y_onehot=enc.transform(y)
    class_names=y_onehot.columns
    X_obj=X.select_dtypes('object')
    X_t_obj=X_t.select_dtypes('object')
    X=X.select_dtypes(exclude='object')
    X_t=X_t.select_dtypes(exclude='object') 
    for class_ in class_names:
        enc=ce.CatBoostEncoder()
        enc.fit(X_obj,y_onehot[class_])
        temp=enc.transform(X_obj)
        temp_t=enc.transform(X_t_obj)
        temp.columns=[str(x)+'_'+str(class_) for x in temp.columns]
        temp_t.columns=[str(x)+'_'+str(class_) for x in temp_t.columns]
        X=pd.concat([X,temp],axis=1)
        X_t=pd.concat([X_t,temp_t],axis=1)
      
    return X, X_t

In [ ]:
X_train, X_test = catboost_encode_multiclass(X_train,X_test,y_train)

In [ ]:
X_train.shape, X_test.shape

((262, 201), (175, 201))

### Model_Baseline

In [ ]:
from sklearn.feature_selection import SelectPercentile
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, ShuffleSplit, StratifiedKFold

In [ ]:
from sklearn.utils import all_estimators
estimators = all_estimators(type_filter='classifier')
names = []
name_list = []
for name, ClassifierClass in estimators:
    names.append(name)
    name_list.append(name)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import CategoricalNB
from sklearn.multioutput import ClassifierChain
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process.gpc import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neural_network.multilayer_perceptron import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestCentroid
from sklearn.svm import NuSVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [ ]:
# for i in range(len(name_list)):
#   name_list[i] = eval(name_list[i])

In [ ]:
names[40:]

In [ ]:
best_parm = []
best_avg = []
for i in range(5):
  # 사용할 모델 설정 (속도가 빠른 모델 사용 권장). 다른 모델 가능
  model = CatBoostClassifier(verbose=False)

  # 각 특성과 타깃(class) 사이에 유의한 통계적 관계가 있는지 계산하여 특성을 선택하는 방법 
  cv_scores = []
  for p in tqdm_notebook(range(10,100,1)):
      X_new = SelectPercentile(percentile=p).fit_transform(X_train, y_train)
      sscv = ShuffleSplit(test_size=.6, n_splits=10)    
      cv_score = cross_val_score(model, X_new, y_train, scoring='f1_macro', cv=sscv).mean()
      cv_scores.append((p,cv_score))

  # Print the best percentile
  best_score = cv_scores[np.argmax([score for _, score in cv_scores])]
  print(best_score)
  best_parm.append(best_score[0])
  best_avg.append(best_score[1])
best_mean = np.mean(best_avg)
print('F1_score_mean:', best_mean, '/', 'F1_score_best:', np.max(best_avg),  '/', 'Best p:', best_parm[best_avg.index(np.max(best_avg))])

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/90 [00:00<?, ?it/s]

(58, 0.9566588640922624)


This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/90 [00:00<?, ?it/s]

...

  0%|          | 0/90 [00:00<?, ?it/s]

(68, 0.9576146136101054)


This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/90 [00:00<?, ?it/s]

(71, 0.9561446874882348)
F1_score_mean: 0.957142760292015 / F1_score_best: 0.9579548694292599 / Best p: 81


In [ ]:
'BaggingClassifier', F1_score_mean: 0.9494594528732219 / F1_score_best: 0.9511554435812538 / Best p: 79
'CalibratedClassifierCV', F1_score_mean: 0.9626664141786619 / F1_score_best: 0.9660142756475478 / Best p: 78
'ExtraTreesClassifier', F1_score_mean: 0.9616226459740507 / F1_score_best: 0.9648649222960971 / Best p: 67
'GaussianNB', F1_score_mean: 0.9500959138856926 / F1_score_best: 0.9534295776429245 / Best p: 98
'GaussianProcessClassifier', F1_score_mean: 0.9648194841883597 / F1_score_best: 0.9679756461684168 / Best p: 34
'GradientBoostingClassifier', F1_score_mean: 0.9501056420017486 / F1_score_best: 0.9531256516510341 / Best p: 80
'HistGradientBoostingClassifier', F1_score_mean: 0.953962663860365 / F1_score_best: 0.9573744770487675 / Best p: 93
'KNeighborsClassifier', F1_score_mean: 0.9615118448606774 / F1_score_best: 0.9693338277851933 / Best p: 45
'LinearSVC', F1_score_mean: 0.9630658893555746 / F1_score_best: 0.9684577625178417 / Best p: 28
'LogisticRegression', F1_score_mean: 0.96801434411589 / F1_score_best: 0.9706982533538591 / Best p: 71
'MLPClassifier', F1_score_mean: 0.9671467283897688 / F1_score_best: 0.9710417461014795 / Best p: 51
'PassiveAggressiveClassifier', F1_score_mean: 0.965491391129117 / F1_score_best: 0.9678124157810576 / Best p: 63
'Perceptron', F1_score_mean: 0.9652085976997506 / F1_score_best: 0.9706022400830374 / Best p: 65
'RidgeClassifier', F1_score_mean: 0.9681223952413746 / F1_score_best: 0.9719497835210567 / Best p: 80
'RidgeClassifierCV', F1_score_mean: 0.9684216195308422 / F1_score_best: 0.9735312766014588 / Best p: 93
'NuSVC', (46, 0.9655154108275946), (65, 0.9640139279073164)
'SGDClassifier', F1_score_mean: 0.965160109064847 / F1_score_best: 0.969927365408479 / Best p: 93
'NearestCentroid', F1_score_mean: 0.9601919676635571 / F1_score_best: 0.9623609475632385 / Best p: 80
'SVC', F1_score_mean: 0.9656216171260938 / F1_score_best: 0.9677177957871897 / Best p: 77
'RandomForestClassifier', F1_score_mean: 0.9617615644836773 / F1_score_best: 0.96616115576859 / Best p: 98
'LabelPropagation', F1_score_mean: 0.9481399399536056 / F1_score_best: 0.9523186698498195 / Best p: 20
'LabelSpreading', F1_score_mean: 0.9478608514893777 / F1_score_best: 0.9516689894453961 / Best p: 31
'LGBMClassifier', F1_score_mean: 0.9544113572783823 / F1_score_best: 0.9574919877922842 / Best p: 64
'XGBClassifier', F1_score_mean: 0.9524315464122562 / F1_score_best: 0.9567336494803451 / Best p: 91
'CatBoostClassifier', F1_score_mean: 0.957142760292015 / F1_score_best: 0.9579548694292599 / Best p: 81

'DecisionTreeClassifier', F1_score_mean: 0.9328998267559822 / F1_score_best: 0.9353486611281709 / Best p: 48
'AdaBoostClassifier', F1_score_mean: 0.9288626025107073 / F1_score_best: 0.9342413721737758 / Best p: 60
'ExtraTreeClassifier', F1_score_mean: 0.9119928779475485 / F1_score_best: 0.9152381232996504 / Best p: 78

'QuadraticDiscriminantAnalysis', F1_score_mean: 0.8753889371841685 / F1_score_best: 0.9084000438461238 / Best p: 16
'LinearDiscriminantAnalysis', F1_score_mean: 0.8714478666180894 / F1_score_best: 0.907537069411009 / Best p: 48
'BernoulliNB', F1_score_mean: 0.7363025685950964 / F1_score_best: 0.7480165172515819 / Best p: 84
'DummyClassifier', F1_score_mean: 0.3561997462115568 / F1_score_best: 0.36252507431486375 / Best p: 92


'CategoricalNB', error
'ClassifierChain', error
'ComplementNB', error
'StackingClassifier', error
'VotingClassifier', error
'RadiusNeighborsClassifier', error
'OneVsOneClassifier', error
'OneVsRestClassifier', error
'OutputCodeClassifier', error
'MultiOutputClassifier', error
'MultinomialNB', error
'LogisticRegressionCV', error

SyntaxError: ignored

In [ ]:
# fs = SelectPercentile(percentile=best_score[0]).fit(X_train, y_train)
# X_train = fs.transform(X_train)
# X_test = fs.transform(X_test)
# X_train = pd.DataFrame(X_train)
# X_test = pd.DataFrame(X_test)

## Submission

In [ ]:
# X_train.to_csv('/content/drive/MyDrive/유전체공모전/data/X_train_select.csv', index=False)
# X_test.to_csv('/content/drive/MyDrive/유전체공모전/data/X_test_select.csv', index=False)